# Implementing Simplified DES (SDES) with SAGE

In [93]:
# Note: make sure to run this cell first
import ipywidgets as widgets

In [94]:
widgets.Image(
    value=open("sdes.png","rb").read(),
    width='100%'
)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0c\x9c\x00\x00\t#\x08\x06\x00\x00\x00\xfe\xa6\xb4\x…

# 1 Definitions and Helper Methods

## 1.a TODO: Expansions and Permutations

In [95]:
# The Expansions/Permutations are stored as lists of bit positions

# TODO
# Please replace the "*" according to the 
# description of S-DES (see "S-DES Handout.pdf"
# as well as appendix G.4, "Simplified DES").
P10_data = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6];
P8_data = [6, 3, 7, 4, 8, 5, 10, 9];
LS1_data = [2, 3, 4, 5, 1];
LS2_data = [3, 4, 5, 1, 2];
IP_data = [2, 6, 3, 1, 4, 8, 5, 7];
IPinv_data = [4, 1, 3, 5, 7, 2, 8, 6];
EP_data = [4, 1, 2, 3, 2, 3, 4, 1];
P4_data = [2, 4, 3, 1];
SW_data = [5, 6, 7, 8, 1, 2, 3, 4];

## 1.b TODO: S-Boxes

In [96]:
# SDES lookup tables

# TODO
# Please replace the "*" with numbers from 0-3, 
# according to the description of S-DES (see p.7
# in appendix G.4, "Simplified DES").
S0_data = [[1, 0, 3, 2],
           [3, 2, 1, 0],
           [0, 2, 1, 3],
           [3, 1, 3, 2]];

S1_data = [[0, 1, 2, 3],
           [2, 0, 1, 3],
           [3, 0, 1, 0],
           [2, 1, 0, 3]];

## 1.c Permutation Helper Method

In [97]:
def ApplyPermutation(X, permutation):
    r"""
    This function takes a permutation list (list of bit positions.)
    And outputs a bit list with the bits taken from X.
    """
    return list(X[i-1] for i in permutation)

## 1.d Substitution Helper Method

In [98]:
def ApplySBox(X, SBox):
    r"""
    This function Applies the SDES SBox (by table look up)
    """
    r = 2*X[0] + X[3];
    c = 2*X[1] + X[2];
    o = SBox[r][c]; 
    return [o & 2, o & 1];

## 1.e Compute Permutations

In [99]:
#
# Each of these functions uses ApplyPermutation
# and a permutation list to perform an SDES
# Expansion/Permutation
#
def P10(X):
    return ApplyPermutation(X, P10_data);

def P8(X):
    return ApplyPermutation(X, P8_data);

def IP(X):
    return ApplyPermutation(X, IP_data);

def IPinv(X):
    return ApplyPermutation(X, IPinv_data);

def EP(X):
    return ApplyPermutation(X, EP_data);

def P4(X):
    return ApplyPermutation(X, P4_data);

def SW(X): 
    return ApplyPermutation(X, SW_data);

def LS1(X):
    return ApplyPermutation(X, LS1_data);

def LS2(X):
    return ApplyPermutation(X, LS2_data);

## 1.f Compute S-Boxes

In [100]:
# These two functions perform the SBox substitutions
def S0(X):
    return ApplySBox(X, S0_data);

def S1(X):
    return ApplySBox(X, S1_data);

## 1.g Concatenate Two Blocks

In [101]:
def concatenate(left, right):
    r"""
    Joins to bit lists together.
    """
    ret = [left[j] for j in range(len(left))];
    ret.extend(right);
    return ret;

## 1.h Return Left/Right Half Bits of a Block

In [102]:
def LeftHalfBits(block):
    r"""
    Returns the left half bits from block.
    """
    l = len(block);
    return [block[j] for j in range(l/2)];

def RightHalfBits(block):
    r"""
    Returns the right half bits from block.
    """
    l = len(block);
    return [block[j] for j in range(l/2, l)];

## 1.i Compute XOR of Two Blocks

In [103]:
def XorBlock(block1, block2):
    r"""
    Xors two blocks together.
    """
    l = len(block1);
    if (l != len(block2)):
        raise (ValueError, "XorBlock arguments must be same length")
    return [(block1[j]+block2[j]) % 2 for j in range(l)];


# 2 Key Generation

## 2.a TODO: Key Generation

In [104]:
def SDESKeySchedule(K):
    r"""
    Expands an SDES Key (bit list) into the two round keys.
    """
    K_after_P10 = P10(K);
    
    lefthalf = LeftHalfBits(K_after_P10);
    
    righthalf = RightHalfBits(K_after_P10);
    
    ls1_l, ls1_r=LS1(lefthalf), LS1(righthalf)
    #print(ls1_l, ls1_r)
    K1=P8(concatenate(left=ls1_l, right=ls1_r))
    
    K2=P8(concatenate(LS2(ls1_l), LS2(ls1_r)))
    
    
    
    #print(f"K1= {K1}, K2={K2}")
    
    
    
    return (K1,K2)

In [105]:
SDESKeySchedule([1,1,1,0,0,0,1,1,0,1])

([1, 1, 0, 0, 1, 1, 1, 0], [1, 1, 0, 0, 1, 1, 0, 1])

## 2.b TODO: Compute F_K function

In [106]:
def f_K(block, K):
    r"""
    Performs the f_K function supplied block and K.
    """
    left = LeftHalfBits(block);
    right= RightHalfBits(block);
    ep=EP(right)
    
    
    s0, s1=S0(LeftHalfBits(XorBlock(K, ep))), S1(RightHalfBits(XorBlock(K, ep)))
    p4=P4(concatenate(s0,s1))
    my_outputblock=concatenate(XorBlock(left, p4), right)
    
    return my_outputblock;

In [107]:
f_K([0,0,0,0, 0,0,0,0],[0,0,0,0, 0,0,0,0])

[1, 0, 0, 0, 0, 0, 0, 0]

# 3 Encryption

## 3.a TODO: Encrypting with SDES

In [108]:
def SDESEncrypt(plaintext_block, K):
    r"""
    Performs a single SDES plaintext block encryption.
    (Given plaintext and key as bit lists.)
    """
    (K1, K2) = SDESKeySchedule(K)
    
    ip=IP(plaintext_block)
    
    out1=f_K(ip, K1)
    
    out2=SW(out1)
    
    out3=f_K(out2, K2)
    
    my_outputblock=IPinv(out3)
    
    
    return my_outputblock;   

# 4 Decryption

## 4.a TODO: Decrypting with SDES

In [109]:
def SDESDecrypt(ciphertext, K):
    r"""
    Performs a single SDES ciphertext block decryption.
    (Given ciphertext and key as bit lists.)
    """
    (K1, K2) = SDESKeySchedule(K);
    ip=IP(ciphertext)
    out1=f_K(ip, K2)
    out2=SW(out1)
    out3=f_K(out2, K1)
    my_outputblock=IPinv(out3)
    
    
    return my_outputblock;

# 5 Testing Implementation

## 5.a TODO: Test your implementation

In [111]:
plaintext = [0, 1, 1, 0, 1, 1, 0, 1]
print("Plaintextt: " + str(plaintext))

K = [1,1,1,0,0,0,1,1,0,1]


ciphertext = SDESEncrypt(plaintext, K)
print("Ciphertext: " + str(ciphertext))

pp = SDESDecrypt(ciphertext, K)
print("Plaintext: " + str(pp))
print("Decrypted cipher text equals to plain text: " + str(pp == plaintext))

Plaintextt: [0, 1, 1, 0, 1, 1, 0, 1]
Ciphertext: [0, 1, 1, 1, 1, 0, 0, 0]
Plaintext: [0, 1, 1, 0, 1, 1, 0, 1]
Decrypted cipher text equals to plain text: True


## 3-DES

In [116]:


K2, K3=[1,1,1,0,0,0,1,1,0,1], [0,1,0,0,0,0,0,1,0,0]
ciphertext = SDESEncrypt(plaintext, K)
c2=SDESDecrypt(ciphertext, K2)
c3=SDESEncrypt(c2, K3)
print(plaintext)
print(c3)
c1=SDESDecrypt(c3, K3)
c2=SDESEncrypt(c1, K2)
pp2=SDESDecrypt(c2, K)


print("Decrypted cipher text equals to plain text: " + str(pp == pp2))


[0, 1, 1, 0, 1, 1, 0, 1]
[0, 0, 1, 1, 1, 1, 0, 1]
Decrypted cipher text equals to plain text: True
